# Análisis de Sentimientos a Nivel de Texto: Aplicación Final
---

> **Proyecto final de Asignatura Sistemas Computacionales <br>
Escuela de Ingeniería de Sistemas <br>
Universidad de Los Andes <br>
Autor: Jhonathan Abreu <br>**

---
<br>
La función de este notebook es la presentación de una aplicación que muestre la utilización del modelo generado en el notebook `modelo.ipynb`. Con esto se pretente probar el clasificador, al predecir el sentimiento de un conjunto de oraciones distintas a las que se utilizaron para el entrenamiento y prueba del modelo.

<br>

## Configuración del sistema de archivos de Google Drive
---

En primer lugar, se debe configurar el sistema de archivos de Google Drive, si va a utilizar Colaboratory:

In [2]:
# Instalar la biblioteca FUSE (Filesystem in Userspase) para manejar el sistema
# de archivos de Google Drive
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties \
    module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Importar bibliotecas necesarias para autenticación
from google.colab import auth
from oauth2client.client import GoogleCredentials
import getpass

# Generar los tokens de autorización para Colaboratory
auth.authenticate_user()

# Generar credenciales para la biblioteca FUSE
credentials = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={credentials.client_id} \
    -secret={credentials.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={credentials.client_id} \
    -secret={credentials.client_secret}

# Montar sistema de archivos
!fusermount -u drive
!sshfs -u drive

# Crear un directorio y montar Google Drive usando ese directorio
!mkdir -p drive
!google-drive-ocamlfuse drive

# Prueba: mostrar contenido de un directorio en drive
print ('Archivos en Drive:')
!ls drive


gpg: keybox '/tmp/tmpacny_y2v/pubring.gpg' created
gpg: /tmp/tmpacny_y2v/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
/bin/sh: 1: sshfs: not found
Archivos en Drive:
aplicacion.ipynb  mejormodelo.ipynb  modelos
datasets	  modelo.ipynb	     preprocesamiento.ipynb


<br>

## Declaración de constantes
---

Las siguientes constantes son necesarias para ubicar los datasets y los modelos. Modifique según sea necesario, si va a utilizar Colaboratory o un entorno local.

In [1]:
import os

# Directorios de los datasets

#   NOTA: cambiar PROJECT_DIR al directorio raíz del proyecto
#     Para drive, cambie la siguiente variable a la ruta del
#     proyecto en su Drive:
#PROJECT_DIR = 'drive/ULA/sistemascomputacionales/aplicacion'
#     Para entorno local, cambie la siguiente variable para apunta
#     a la ruta abosluta del proyecto en su disco:
PROJECT_DIR = ('/home/jhonathanabreu/Documentos/ula/sistemas_computacionales/'
               'Proyecto_SistemasComputacionales_AbreuJ/aplicacion')
DATASETS_DIR = os.path.join(PROJECT_DIR, 'datasets')
ORIGINAL_DATASETS_DIR = os.path.join(DATASETS_DIR, 'originales')
PROCESSED_DATASETS_DIR = os.path.join(DATASETS_DIR, 'procesados')
EVALUATION_DATASETS_DIR = os.path.join(DATASETS_DIR, 'evaluacion')
MODELS_DIR = os.path.join(PROJECT_DIR, 'modelos')

<br>
## Preparación del entorno y funciones para el vectorizador
---

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer

import numpy as np

from string import punctuation

# Descarga y carga de la lista de palabras vacías de NLTK

nltk.download('punkt')

nltk.download('stopwords')
spanishStopWords = stopwords.words('spanish')

# Carga y extensión de la lista de signos de puntuación y otros símbolos.

nonWords = list(punctuation)
nonWords.extend(['¿', '¡'])  # Se agregan estos símbolos (español)
nonWords.extend(map(str,range(10)))  # Se agregan los dígitos numéricos

# Stemmer, objeto que llevará las palabras a sus raíces 
stemmer = SnowballStemmer('spanish')

# Función que aplica el stemming
def stem_tokens(tokens, stemmer):
    stemmedTokens = []
    for token in tokens:
        stemmedTokens.append(stemmer.stem(token))
        
    return stemmedTokens

# Función que limpia y tokeniza las frases
def tokenize(text):
    # Eliminación de símbolos y números
    text = ''.join([c for c in text if c not in nonWords])
    # Tokeninazión
    tokens =  word_tokenize(text)

    # Stemming
    try:
        stemmedTokens = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stemmedTokens = ['']
        
    return stemmedTokens

[nltk_data] Downloading package punkt to
[nltk_data]     /home/jhonathanabreu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jhonathanabreu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<br>
## Predictor de Sentimientos
---

### Carga del vectorizador

In [3]:
from sklearn.externals import joblib

def loadVectorizer(self, vectorizerFileName):
    vectorizer = joblib.load(vectorizerFileName)
    
    # Cálculo de la dimensión de los vectores
    sentence = 'Test sentence for calculating vectors dimension'
    sentenceVector = vectorizer.transform([sentence])
    
    self.vectorsDimension = sentenceVector.shape[1]
    
    return vectorizer

### Carga del clasificador

Primero, vamos a **definir el modelo** como en el notebook `modelo.ipynb`:

In [13]:
import numpy as np 
import pandas as pd

pd.set_option('display.max_colwidth', -1)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D

def buildModel(self, inputDimension):
    model = Sequential()
    
    model.add(Conv1D(128, 1, input_shape = (1, inputDimension),
                     activation = 'relu'))
    model.add(Conv1D(32, 1, activation = 'relu'))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))    
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

    return model

<br>
El siguiente paso es la **carga del modelo:**

In [14]:
def loadClassifier(self, classifierFileName):
    model = self.buildModel(self.vectorsDimension)
    model.load_weights(classifierFileName)
    
    return model

### Preparación de la data de entrada

In [15]:
def prepareInput(self, inputData):
    vectors = self.vectorizer.transform(inputData).toarray()
    vectors = vectors.reshape(vectors.shape[0], 1, vectors.shape[1])
        
    return vectors

### Predición (clasificación) de sentimientos

In [16]:
def predict(self, sentences):
    if isinstance(sentences, str):
        sentences = [sentences]
    else:
        try:
           _ = (e for e in sentences)
        except TypeError:
            # No es un iterable
            return None
        
    vectors = self.prepareInput(sentences)
    
    probabilities = self.classifier.predict(vectors)
    sentiments = [self.sentimentLabels[np.argmax(sentimentProbabilities)]
                  for sentimentProbabilities in probabilities]
    
    dataFrameItems = [('Oración', sentences),
                      ('Sentimiento', sentiments)]
    
    return pd.DataFrame.from_items(dataFrameItems)
    

### La clase final

In [17]:
class SentimentPredictor:
    
    sentimentLabels = {
        0: 'Negativo',
        1: 'Neutral',
        2: 'Positivo'
    }
    
    vectorsDimension = 0
    
    def __init__(self, vectorizerFilename, classifierFilename):
        self.vectorizerFilename = vectorizerFilename
        self.classifierFilename = classifierFilename
        
        # Carga del vectorizador
        self.vectorizer = self.loadVectorizer(self.vectorizerFilename)
        
        # Carga del clasificador
        self.classifier = self.loadClassifier(self.classifierFilename)
        
        
    # Métodos de la clase
    
    loadVectorizer = loadVectorizer
    
    buildModel = buildModel
    
    loadClassifier = loadClassifier
    
    prepareInput = prepareInput
    
    predict = predict
    

<br>
## Clasificando oraciones
---

### Carga e inicialización del modelo

In [18]:
vectorizerFilename = os.path.join(MODELS_DIR, 'vectorizador.pkl')
classifierFilename = os.path.join(MODELS_DIR, 'modelo.hdf5')

classifier = SentimentPredictor(vectorizerFilename, classifierFilename)

### Pedicción

El archivo `oraciones.txt` contiene algunas frase para probar. Añada oraciones, una en cada línea, si desea probar otras diferentes.

In [20]:
sentencesFilename = os.path.join(EVALUATION_DATASETS_DIR, 'oraciones.txt')
with open(sentencesFilename) as file:
    sentences = file.read().splitlines()

predictions = classifier.predict(sentences)

display(predictions)

,Oración,Sentimiento
0,Hoy será un gran día pues voy a alcanzar todas mis metas,Positivo
1,Que terrible es la comida que preparas,Neutral
2,"Todos tus sueños se volverán realidad, solo ten fe en ti mismo",Positivo
3,"Mejor temprano que tarde, mejor tarde que nunca",Positivo
4,Eres la mejor persona del mundo mundial,Positivo
5,Soy una persona muy inutil,Negativo
6,Tengo mucha hambre y sueño,Positivo
7,No tengo nada que decir,Neutral
8,Eres una persona terriblemente malvada y mala,Negativo
9,Tu trabajo no sirve y es malo,Negativo
